In [0]:
from pyspark.sql.functions import col, lead, broadcast, collect_set, size, array_contains
from pyspark.sql.window import Window

class Transformer:
    def __init__(self):
        pass

    def transform(self, input_df):
        pass

class TransformerOperations(Transformer):
    # 1. Customers who have bought AirPods after purchasing iPhone
    def airpord_after_iphone_transform(self, input_dfs):
        transaction_df = input_dfs.get("transaction_input_df")
        print("TransformerOperations / airpord_after_iphone_transform / transaction_input_df")
        transaction_df = transaction_df \
            .withColumn("transaction_id", col("transaction_id").cast("int")) \
            .withColumn("transaction_date", col("transaction_date").cast("date"))
        
        #transaction_df.show()
        #transaction_df.printSchema()

        transformed_df = transaction_df \
            .withColumn("next_product_name", lead("product_name").over(Window.partitionBy("customer_id").orderBy("transaction_date")))
        
        #transformed_df.orderBy("customer_id", "transaction_date").show()
        filtered_df = transformed_df.filter( (col("product_name") == "iPhone") & (col("next_product_name") == "AirPods") )

        filtered_df.show()

        customer_df = input_dfs.get("customer_input_df")
        
        join_df = customer_df.join(broadcast(filtered_df), "customer_id", "inner") \
            .select(filtered_df["customer_id"], "customer_name", "location")

        join_df.show()
        return join_df
    

    def airpods_and_iphone_transform(self, input_dfs):
        """
        Generate data for all customers who bought both Airpods + iPhone products only and no other products
        """
        transaction_df = input_dfs.get("transaction_input_df")
        print("TransformerOperations / airpods_and_iphone_transform / transaction_input_df")

        grouped_df = transaction_df.groupBy("customer_id").agg(collect_set("product_name").alias("products"))
        
        print("TransformerOperations / airpods_and_iphone_transform / grouped_df")
        grouped_df.show(truncate=False)

        filtered_df = grouped_df \
            .filter( 
                    (array_contains(col("products"), "iPhone")) & \
                    (array_contains(col("products"), "AirPods")) & \
                    (size(col("products")) == 2) \
            )

        print("TransformerOperations / airpods_and_iphone_transform / filtered_df")
        filtered_df.show()

        customer_df = input_dfs.get("customer_input_df")
        join_df = customer_df.join(broadcast(filtered_df), "customer_id", "inner") \
            .select(filtered_df["customer_id"], "customer_name", "location")

        join_df.show()
        return join_df